## Columbia University
### ECBM E4040 Neural Networks and Deep Learning. Fall 2021.

# ECBM E4040 - Assignment 2- Task 5: Kaggle Open-ended Competition

Kaggle is a platform for predictive modelling and analytics competitions in which companies and researchers post data and statisticians and data miners compete to produce the best models for predicting and describing the data.

If you don't have a Kaggle account, feel free to join at [www.kaggle.com](https://www.kaggle.com). To let the CAs do the grading more conveniently, please __use Lionmail to join Kaggle__ and __use UNI as your username__.

The competition is located here: https://www.kaggle.com/c/ecbm4040-assignment-2-task-5/overview.

You can find detailed description about this in-class competition on the website above. Please read carefully and follow the instructions.

<span style="color:red">__TODO__:</span>
1. Train a custom model for the bottle dataset classification problem. You are free to use any methods taught in the class or found by yourself on the Internet (ALWAYS provide reference to the source).
General training methods include:
    * Dropout
    * Batch normalization
    * Early stopping
    * l1-norm & l2-norm penalization
    
2. You are given the test set to generate your predictions (70% public + 30% private, but you don't know which ones are public/private). Students should achieve an accuracy on the public test set of at least 70%. Two points will be deducted for each 1% below 70% accuracy threshold (i.e. 65% accuracy will have 10 points deducted). The accuracy will be shown on the public leaderboard once you submit your prediction .csv file. The private leaderboard will be released after the competition. The final ranking is based on the private leaderboard result, not the public leaderboard.
3. 

    * Report your results on the Kaggle, for comparison with other students' optimal results (you can do this several times). 
    * Save your best model, using Github Classroom, at the same time when you submit the homework files into Courseworks. See instructions below. 

__Hint__: You can start from what you implemented in task 4. Another classic classification model named 'VGG16' can also be easily implemented. Students are allowed to use pretrained networks, and utilize transfer learning. 

## HW Submission Details:
There are three components to reporting the results of this task: 

**(A) Submission (up to 20 submissions each day) of the .csv prediction file throught the Kaggle platform;**. You should start doing this __VERY early__, so that students can compare their work as they are making progress with model optimization.

**(B) Editing and submitting the content of this Jupyter notebook, through Courseworks; **
(i) The code for your CNN model and for the training function. The code should be stored in __./ecbm4040/neuralnets/kaggle.py__;
(ii) Print out your training process and accuracy __within this notebook__;

**(C) Submitting your best CNN model through Github Classroom repo.**

**Description of (C):** 
For this task, we will continue to use Github classroom to save your model for submission. 

<span style="color:red">__Submission content:__ :</span>
(i) In your Assignment 2 submission folder, create a subfolder called __KaggleModel__. Upload your best model with all the data output (for example, __MODEL.data-00000-of-00001, MODEL.meta, MODEL.index__) into the folder. 
(ii) Remember to delete any intermediate results, **we only want your best model. Do not upload any data files**. The instructors will rerun the uploaded best model and verify against the score which you reported on the Kaggle.



## Load Data

In [3]:
import os
!pip install kaggle
os.environ['KAGGLE_USERNAME'] = "kateharwood"
os.environ['KAGGLE_KEY'] = "2190c2afe8f72ccb558fc42670c0ed9a"


You should consider upgrading via the '/home/ecbm4040/envTF24/bin/python3 -m pip install --upgrade pip' command.


In [4]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()
!kaggle competitions download -c ecbm4040-assignment-2-task-5

import zipfile
with zipfile.ZipFile("ecbm4040-assignment-2-task-5.zip", 'r') as zip_ref:
    zip_ref.extractall("kaggle_data")


 99%|████████████████████████████████████████▍| 481M/487M [00:04<00:00, 138MB/s]
100%|█████████████████████████████████████████| 487M/487M [00:04<00:00, 120MB/s]


In [1]:
#Generate dataset
import os
import pandas as pd
import numpy as np
from PIL import Image


#Load Training images and labels
train_directory = os.getcwd() + "/kaggle_data/kaggle_train_128/train_128" #TODO: Enter path for train128 folder (hint: use os.getcwd())
image_list=[]
label_list=[]
for sub_dir in os.listdir(train_directory):
    print("Reading folder {}".format(sub_dir))
    sub_dir_name=os.path.join(train_directory,sub_dir)
    for file in os.listdir(sub_dir_name):
        filename = os.fsdecode(file)
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_list.append(np.array(Image.open(os.path.join(sub_dir_name,file))))
            label_list.append(int(sub_dir))
X_train=np.array(image_list)
y_train=np.array(label_list)

#Load Test images
test_directory = os.getcwd() + "/kaggle_data/kaggle_test_128/test_128"#TODO: Enter path for test128 folder (hint: use os.getcwd())
test_image_list=[]
test_df = pd.DataFrame([], columns=['Id', 'X'])
print("Reading Test Images")
for file in os.listdir(test_directory):
    filename = os.fsdecode(file)
    if filename.endswith(".jpg") or filename.endswith(".png"):
        test_df = test_df.append({
            'Id': filename,
            'X': np.array(Image.open(os.path.join(test_directory,file)))
        }, ignore_index=True)
        
test_df['s'] = [int(x.split('.')[0]) for x in test_df['Id']]
test_df = test_df.sort_values(by=['s'])
test_df = test_df.drop(columns=['s'])
X_test = np.stack(test_df['X'])


print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)

Reading folder 1
Reading folder 3
Reading folder 0
Reading folder 2
Reading folder 4
Reading Test Images
Training data shape:  (15000, 128, 128, 3)
Training labels shape:  (15000,)
Test data shape:  (3500, 128, 128, 3)


In [31]:
# Split data into training and validation
# Not used for final run so as to get the most out of the training data
# X_val = X_train[12000:]
# y_val = y_train[12000:]
# X_train = X_train[:12000]
# y_train = y_train[:12000]

In [2]:
# Can run it from the kaggle.py file or from in the notebook
from utils.neuralnets.kaggle import train_model

model = train_model(X_train, y_train)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_128 (Function (None, 4, 4, 1024)        3228864   
_________________________________________________________________
dropout (Dropout)            (None, 4, 4, 1024)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 5)                 81925     
Total params: 3,310,789
Trainable params: 3,288,901
Non-trainable params: 21,888
_________________________________________________________________
Epoch 1/6
108/108 [==============================] - 34s 257ms/step - loss: 1.6671 - accuracy: 0.7151
Epoch 2/6
108/108 [==============================] - 27s 251ms/step - loss: 0.2517 - accuracy: 0.9523
Epoch 3/6
108/108 [================

## Build and Train Your Model Here

In [2]:
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
import datetime

def create_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(kernel_regularizer=tf.keras.regularizers.l2(0.01), filters=6, kernel_size=(5, 5), strides=(1,1), activation='tanh', input_shape=X_train.shape[1:4], padding="same"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(5, activation='softmax'))

    return model

def create_transfer_learning_model():
    base_model = tf.keras.applications.MobileNet(input_shape=X_train.shape[1:4],
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = True
    
    model = tf.keras.models.Sequential()
    model.add(base_model)
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(5, activation='softmax'))
    
    return model

model_test = create_transfer_learning_model()
model_test.summary()
model_test.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_test.fit(x=X_train, 
          y=y_train,
          batch_size=140, #140
          epochs=6) #100 (for my own network)
#           validation_data=(X_val, y_val)) 


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_128 (Function (None, 4, 4, 1024)        3228864   
_________________________________________________________________
dropout (Dropout)            (None, 4, 4, 1024)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 5)                 81925     
Total params: 3,310,789
Trainable params: 3,288,901
Non-trainable params: 21,888
_________________________________________________________________
Epoch 1/6
108/108 [==============================] - 33s 250ms/step - loss: 1.7270 - accuracy: 0.7215
Epoch 2/6
108/108 [==============================] - 27s 253ms/step - loss: 0.2352 - accuracy: 0.9569
Epoch 3/6
108/108 [================

## Save your best model

In [10]:
model_test.save(filepath = "./model/KaggleModel")

INFO:tensorflow:Assets written to: ./model/KaggleModel/assets


In [9]:
saved_model = tf.keras.models.load_model("./model/KaggleModel")

In [3]:
output = model_test.predict(X_test)
predictions = np.argmax(output, axis=-1)

## Generate .csv file for Kaggle

The following code snippet can be used to generate your prediction .csv file.

NOTE: If your kaggle results are indicating random performance, then it's likely that the indices of your csv predictions are misaligned.

In [4]:
import csv
with open('predicted.csv','w') as csvfile:
    fieldnames = ['Id','label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for index,l in enumerate(predictions):
        filename = str(index) + '.png'
        label = str(l)
        writer.writerow({'Id': filename, 'label': label})